In [4]:
import tensorflow as tf
from math import *
import mnist_reader

X_test, y_test = mnist_reader.load_mnist('../../data/fashion', kind='t10k')
X_train, y_train = mnist_reader.load_mnist('../../data/fashion', kind='train')

HEIGHT = 28
WIDTH = 28
DATA_NUM = 60000
INDEX = 200
num = INDEX
k =11

#欧式距离
def o_distance(coordinateSet):
    square_sum = list(map(lambda x: (x[0]-x[1])**2,coordinateSet))
    for sq in range(len(square_sum)):
        square_sum[sq] = sqrt(fsum(square_sum[sq]))
    return square_sum

#获取欧氏距离有序表，图像标签表，欧氏距离无序表
def run(input_img_data, img_label_list, od_k_min):
    coordinateSet = [(input_img_data,y,z) for y,z in img_label_list]
    o_distance_list = o_distance(coordinateSet)
    od_k_min = sorted(o_distance_list)
    return od_k_min, coordinateSet, o_distance_list

def readTFR(DATA_NUM):
    filename_queue = tf.train.string_input_producer(['fashion_mnist.tfrecords'])
    reader = tf.TFRecordReader()

    _, example = reader.read(filename_queue)
    features = tf.parse_single_example(
        example,
        features={
            'label':tf.FixedLenFeature([], tf.int64),
            'image_raw':tf.FixedLenFeature([], tf.string)
        }
    )
    image = tf.decode_raw(features['image_raw'], tf.uint8) 
    image = tf.reshape(image, [28, 28, 1])
    label = tf.cast(features['label'], tf.int32)  

    image_batch, label_batch = tf.train.shuffle_batch([image, label],  
                                                    batch_size=1,  
                                                    capacity=100,  
                                                    num_threads=2,
                                                    min_after_dequeue=50)  
    image = tf.reshape(image_batch, [HEIGHT*WIDTH])

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        img_label_list = []
        for i in range(DATA_NUM):
            img, label = sess.run([image, label_batch])
            #构建图像标签列表
            img_label_list.append((img, label))
    sess.close()
    return img_label_list

#实现KNN算法
def KNN(INDEX, DATA_NUM, num):
    img_label_list = readTFR(DATA_NUM)
    coordinateSet = []
    o_distance_list = []
    print("————————————Starting————————————")
    for index in range(len(X_test[0:INDEX])):
        od_k_min, coordinateSet, o_distance_list = run(X_test[index], img_label_list, o_distance_list)

        #获取索引
        min_index = [o_distance_list.index(min) for min in od_k_min[:k]]
        label_list = [0]*10
        #求k的各类占比
        for m_index in min_index:
            label_list[coordinateSet[m_index][2][0]] += 1
        #分类  
        max_current = max(label_list)
        if label_list.index(max_current) != y_test[index]:
            num -= 1
        print('[进行了]: %d个 [正确率]: %f' % (index+1,num/INDEX))
    print("——————————————End——————————————")

if __name__ == '__main__':
    KNN(INDEX, DATA_NUM, num)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/RandomShuffle)]]————————————Starting————————————

[进行了]: 1个 [正确率]: 0.995000
[进行了]: 2个 [正确率]: 0.995000
[进行了]: 3个 [正确率]: 0.995000
[进行了]: 4个 [正确率]: 0.995000
[进行了]: 5个 [正确率]: 0.990000
[进行了]: 6个 [正确率]: 0.990000
[进行了]: 7个 [正确率]: 0.990000
[进行了]: 8个 [正确率]: 0.985000
[进行了]: 9个 [正确率]: 0.985000
[进行了]: 10个 [正确率]: 0.985000
[进行了]: 11个 [正确率]: 0.985000
[进行了]: 12个 [正确率]: 0.985000
[进行了]: 13个 [正确率]: 0.980000
[进行了]: 14个 [正确率]: 0.980000
[进行了]: 15个 [正确率]: 0.975000
[进行了]: 16个 [正确率]: 0.975000
[进行了]: 17个 [正确率]: 0.975000
[进行了]: 18个 [正确率]: 0.970000
[进行了]: 19个 [正确率]: 0.970000
[进行了]: 20个 [正确率]: 0.970000
[进行了]: 21个 [正确率]: 0.970000
[进行了]: 22个 [正确率]: 0.970000
[进行了]: 23个 [正确率]: 0.